# **Важно!** 

Домашнее задание состоит из нескольких задач, которые вам нужно решить.
*   Баллы выставляются по принципу выполнено/невыполнено.
*   За каждую выполненую задачу вы получаете баллы (количество баллов за задание указано в скобках).

**Инструкция выполнения:** Выполните задания в этом же ноутбуке (места под решения **КАЖДОЙ** задачи обозначены как **#НАЧАЛО ВАШЕГО РЕШЕНИЯ** и **#КОНЕЦ ВАШЕГО РЕШЕНИЯ**)

**Как отправить задание на проверку:** Вам необходимо сохранить ваше решение в данном блокноте и отправить итоговый **файл .IPYNB** на учебной платформе в **стандартную форму сдачи домашнего задания.**

**Срок проверки преподавателем:** домашнее задание проверяется **в течение 3 дней после дедлайна сдачи** с предоставлением обратной связи

# **Прежде чем проверять задания:**

1. Перезапустите **ядро (restart the kernel)**: в меню, выбрать **Ядро (Kernel)**
→ **Перезапустить (Restart)**
2. Затем **Выполнить** **все ячейки (run all cells)**: в меню, выбрать **Ячейка (Cell)**
→ **Запустить все (Run All)**.

После ячеек с заданием следуют ячейки с проверкой **с помощью assert.**

Если в коде есть ошибки, assert выведет уведомление об ошибке.

Если в коде нет ошибок, assert отработает без вывода дополнительной информации.

---

In [ ]:
import math
import re
from random import randrange, shuffle, random, randint
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
"""
Задание 1 (1 балл): Генерация пакетов данных для обучения модели BERT

**Цель задания:** Написать код, который генерирует пакеты данных для обучения модели BERT с использованием Python и библиотеки NumPy.

**Задачи:**
1. Сформировать пустой список `batch` для хранения пакетов данных.
2. Инициализировать счетчики `positive` и `negative` для отслеживания положительных и отрицательных пар предложений.
3. В цикле, пока количество положительных и отрицательных пар в `batch` не достигнет заданного размера (`batch_size/2`), выполнить следующие шаги:
    - Выбрать случайные индексы двух предложений из списка `sentences`.
    - Получить сами предложения по выбранным индексам и сохранить их в переменные `tokens_a` и `tokens_b`.
    - Собрать входные данные для модели BERT, добавив специальные токены `[CLS]` и `[SEP]`. Используйте переменные `input_ids` и `segment_ids` 
    для этой цели.
    - Реализовать маскирование токенов (MASK LM):
        - Определить количество токенов, которые будут маскированы (подставлены вместо них `[MASK]`).
        - Создать список кандидатов для маскирования (`cand_maked_pos`) и перемешать его.
        - Заполнить списки `masked_tokens` и `masked_pos` маскированными токенами и их позициями.
        - С вероятностью 80%, замаскировать токен, а с вероятностью 10%, заменить его случайным словом из словаря.
    - Дополнить последовательности нулями (`0`) до максимальной длины (`maxlen`).
    - Дополнить нулями маскированные токены и их позиции, чтобы их количество соответствовало максимально возможному (`max_pred`).
    - Определить, являются ли два предложения последовательными (`IsNext`) или нет (`NotNext`) на основе индексов предложений и счетчиков `positive`
    и `negative`.
    - Добавить полученные данные в список `batch` как положительную пару (`IsNext`) или отрицательную пару (`NotNext`).
    - Увеличить соответствующий счетчик (`positive` или `negative`) на `1`.
4. Вернуть сформированный список `batch` как результат выполнения функции.

**Примечания:**
- Задания выполняются в рамках задачи Next Sentence Prediction (NSP) для обучения модели BERT.
- Вы можете использовать функции и библиотеки Python, такие как `randrange`, `shuffle`, `random`, `randint`, `extend` и другие, для реализации
задачи.
- Весь необходимый функционал для работы с данными и списками будет предоставлен вам. Ваша задача - реализовать собственно формирование
пакетов данных на основе предоставленного кода и комментариев.

В случае затруднений можно обратиться к:
https://github.com/graykode/nlp-tutorial/blob/master/5-2.BERT/BERT.py
"""

def make_batch():
    # НАЧАЛО ВАШЕГО РЕШЕНИЯ
    raise NotImplementedError() # удалить эту строку в процессе решения
    # КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [ ]:
def get_attn_pad_mask(seq_q, seq_k):
    """
    Функция для создания маски внимания, которая скрывает внимание на токенах-заполнителях (PAD).
    
    Args:
    - seq_q: Тензор с представлением запросов (batch_size x len_q).
    - seq_k: Тензор с представлением ключей (batch_size x len_k).
    
    Returns:
    - pad_attn_mask: Маска для внимания, скрывающая PAD-токены (batch_size x 1 x len_k(=len_q)).
    """
    batch_size, len_q = seq_q.size()
    batch_size, len_k = seq_k.size()
    
    # Создаем маску, в которой значение True (1) будет соответствовать PAD-токенам.
    pad_attn_mask = seq_k.data.eq(0).unsqueeze(1)  # batch_size x 1 x len_k(=len_q), где 1 означает маскирование
    
    # Расширяем маску до размерности batch_size x len_q x len_k, чтобы ее можно было использовать для маскирования внимания.
    return pad_attn_mask.expand(batch_size, len_q, len_k)  # batch_size x len_q x len_k


def gelu(x):
    """
    Реализация активационной функции GELU (Gaussian Error Linear Unit) от Hugging Face.

    Args:
    - x: Входной тензор.

    Returns:
    - Результат применения функции GELU к входному тензору.
    """
    return x * 0.5 * (1.0 + torch.erf(x / math.sqrt(2.0)))

In [ ]:
class Embedding(nn.Module):
    def __init__(self):
        super(Embedding, self).__init__()
        self.tok_embed = nn.Embedding(vocab_size, d_model)  # Токенное встраивание
        self.pos_embed = nn.Embedding(maxlen, d_model)  # Встраивание позиции
        self.seg_embed = nn.Embedding(n_segments, d_model)  # Встраивание сегмента (типа токена)
        self.norm = nn.LayerNorm(d_model)

    def forward(self, x, seg):
        seq_len = x.size(1)
        pos = torch.arange(seq_len, dtype=torch.long)
        pos = pos.unsqueeze(0).expand_as(x)  # (seq_len,) -> (batch_size, seq_len)
        # Общее встраивание, включая токены, позиции и сегменты
        embedding = self.tok_embed(x) + self.pos_embed(pos) + self.seg_embed(seg)
        return self.norm(embedding)

In [ ]:
"""
## Задание 2 (1 балл): Реализация класса ScaledDotProductAttention

**Цель задания:** Написать класс `ScaledDotProductAttention`, который реализует механизм внимания с масштабированным скалярным произведением.

**Задачи:**
1. Создать класс `ScaledDotProductAttention`, который наследует `nn.Module` из библиотеки PyTorch.
2. В методе `forward` класса `ScaledDotProductAttention`, выполнить следующие действия:
   - Вычислите оценки (scores) как скалярное произведение матрицы запросов (Q) и транспонированной матрицы ключей (K). 
   Масштабируйте оценки на обратный квадратный корень от размерности `d_k`.
   - Примените маску внимания (`attn_mask`) для скрытия определенных значений в оценках (scores). 
   Для этого используйте метод `masked_fill_`, который заменяет элементы тензора на заданное значение, где маска равна 1.
   - Примените функцию softmax для вычисления весов внимания, которые определяют, насколько каждый элемент входных данных важен.
   - Вычислите контекст путем взвешенной суммы значений (V) с использованием весов внимания.
   - Верните контекст и веса внимания.

В случае затруднений можно обратиться к:
https://github.com/graykode/nlp-tutorial/blob/master/5-2.BERT/BERT.py
"""

class ScaledDotProductAttention(nn.Module):
    def __init__(self):
        super(ScaledDotProductAttention, self).__init__()

    def forward(self, Q, K, V, attn_mask):
        # НАЧАЛО ВАШЕГО РЕШЕНИЯ
        raise NotImplementedError() # удалить эту строку в процессе решения
        # КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [ ]:
"""
## Задание 3 (1 балл): Реализация класса MultiHeadAttention

**Цель задания:** Написать класс `MultiHeadAttention`, который реализует многоголовое внимание (Multi-Head Attention) в модели трансформера.

**Задачи:**
1. Создать класс `MultiHeadAttention`, который наследует `nn.Module` из библиотеки PyTorch.
2. В конструкторе класса `MultiHeadAttention`, определить следующие атрибуты:
   - `W_Q`: Используйте линейный слой `nn.Linear` для создания преобразования запросов (Q). 
   Размерность выхода должна быть `d_k * n_heads`. Прокомментируйте это как "Преобразование запросов".
   - `W_K`: Используйте линейный слой `nn.Linear` для создания преобразования ключей (K). 
   Размерность выхода должна быть `d_k * n_heads`. Прокомментируйте это как "Преобразование ключей".
   - `W_V`: Используйте линейный слой `nn.Linear` для создания преобразования значений (V). 
   Размерность выхода должна быть `d_v * n_heads`. Прокомментируйте это как "Преобразование значений".
3. В методе `forward` класса `MultiHeadAttention`, выполнить следующие действия:
   - Сохраните оригинальные запросы `Q` и определите размер пакета (batch_size).
   - Преобразуйте запросы, ключи и значения (`Q`, `K`, `V`) с использованием линейных преобразований `W_Q`, `W_K` и `W_V`, 
   учитывая многие головы. Результаты преобразований должны быть разделены на головы и транспонированы.
   - Создайте маску внимания (`attn_mask`) с учетом входной маски `attn_mask` и повторите ее для каждой головы.
   - Примените механизм внимания (`ScaledDotProductAttention`) к головам, передав преобразованные запросы, ключи, значения и маску внимания.
   - Транспонируйте и объедините результаты голов для получения контекста.
   - Примените линейное преобразование и слой нормализации к контексту.
   - Верните нормализованный контекст и веса внимания.

В случае затруднений можно обратиться к:
https://github.com/graykode/nlp-tutorial/blob/master/5-2.BERT/BERT.py
"""

class MultiHeadAttention(nn.Module):
    # НАЧАЛО ВАШЕГО РЕШЕНИЯ
    raise NotImplementedError() # удалить эту строку в процессе решения
    # КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [ ]:
"""
## Задание 4 (1 балл): Реализация класса PoswiseFeedForwardNet

**Цель задания:** Написать класс `PoswiseFeedForwardNet`, который реализует полносвязную нейронную сеть с прямым распространением 
(Feed Forward Neural Network) в модели трансформера.

**Задачи:**
1. Создать класс `PoswiseFeedForwardNet`, который наследует `nn.Module` из библиотеки PyTorch.
2. В конструкторе класса `PoswiseFeedForwardNet`, определить два атрибута:
   - `fc1`: Используйте линейный слой `nn.Linear`, чтобы создать первый линейный слой прямого распространения (feed-forward). 
   Размерность входа должна быть `d_model`, а размерность выхода - `d_ff`. Прокомментируйте это как "Первый линейный слой (прямого распространения)".
   - `fc2`: Используйте линейный слой `nn.Linear`, чтобы создать второй линейный слой прямого распространения. 
   Размерность входа должна быть `d_ff`, а размерность выхода - `d_model`. Прокомментируйте это как "Второй линейный слой (прямого распространения)".
3. В методе `forward` класса `PoswiseFeedForwardNet`, выполнить следующие действия:
   - Пропустите входные данные `x` через первый линейный слой `fc1`.
   - Примените функцию активации GELU (Gaussian Error Linear Unit) к результату первого линейного слоя.
   - Пропустите результат через второй линейный слой `fc2`.
   - Верните полученный результат в качестве выходных данных модели.

В случае затруднений можно обратиться к:
https://github.com/graykode/nlp-tutorial/blob/master/5-2.BERT/BERT.py
"""


class PoswiseFeedForwardNet(nn.Module):
    # НАЧАЛО ВАШЕГО РЕШЕНИЯ
    raise NotImplementedError() # удалить эту строку в процессе решения
    # КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [ ]:
"""
## Задание 5 (1 балл): Реализация класса EncoderLayer

**Цель задания:** Написать класс `EncoderLayer`, который представляет собой один слой кодировщика в модели трансформера.

**Задачи:**
1. Создать класс `EncoderLayer`, который наследует `nn.Module` из библиотеки PyTorch.
2. В конструкторе класса `EncoderLayer`, определить два атрибута:
   - `enc_self_attn`: Используйте класс `MultiHeadAttention`, чтобы создать многоголовое внимание для кодировщика. 
   Прокомментируйте это как "Многоголовое внимание для кодировщика".
   - `pos_ffn`: Используйте класс `PoswiseFeedForwardNet`, чтобы создать полносвязную нейронную сеть с прямым распространением. 
   Прокомментируйте это как "Полносвязная нейронная сеть с прямым распространением".
3. В методе `forward` класса `EncoderLayer`, выполнить следующие действия:
   - Применить многоголовое внимание `enc_self_attn` для самокодирования кодировщика, используя входные данные 
   `enc_inputs` в роли запросов, ключей и значений.
   - Применить полносвязную нейронную сеть с прямым распространением `pos_ffn` для обработки выходных данных многоголового внимания.
   - Вернуть обработанные данные и веса внимания.

В случае затруднений можно обратиться к:
https://github.com/graykode/nlp-tutorial/blob/master/5-2.BERT/BERT.py
"""

class EncoderLayer(nn.Module):
    # НАЧАЛО ВАШЕГО РЕШЕНИЯ
    raise NotImplementedError() # удалить эту строку в процессе решения
    # КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [ ]:
"""
**Задание 6 (1 балл): Реализация модели BERT**

**Цель:** Ваша задача - реализовать модель BERT (Bidirectional Encoder Representations from Transformers) с использованием PyTorch.

**Инструкции:**

1. Ваша задача - реализовать класс BERT.

Описание задания для студентов: Реализация модели BERT

1. Создайте класс `BERT`, который будет наследоваться от `nn.Module`. В этом классе вы должны реализовать архитектуру BERT.

2. В конструкторе класса `BERT`, выполните следующие шаги:
   - Создайте модуль встраивания (Embedding) и добавьте его в класс.
   - Создайте список слоев кодировщика (EncoderLayer) и добавьте его в класс. Количество слоев кодировщика (`n_layers`) можно задать в качестве аргумента конструктора.
   - Создайте линейный слой (`nn.Linear`) для извлечения признаков из первого токена (CLS) и добавьте функцию активации (гиперболический тангенс).
   - Создайте второй линейный слой для извлечения скрытых признаков из выхода трансформера на позициях маскированных токенов. В качестве функции активации используйте GELU (Gaussian Error Linear Unit).
   - Добавьте слой нормализации (LayerNorm) и линейный классификатор для задачи классификации.

3. Реализуйте метод `forward`, который принимает входные данные (`input_ids`, `segment_ids`, `masked_pos`) и выполняет следующие действия:
   - Применяет входные данные через встраивание и кодировщик.
   - Извлекает признаки из первого токена (CLS) для классификации.
   - Извлекает скрытые признаки из выхода трансформера на позициях маскированных токенов.
   - Возвращает результаты классификации и предсказания маскированных токенов.

4. Обратите внимание на использование функций активации, слоев нормализации и линейных слоев в методе `forward`.

5. Создайте декодер, который будет использоваться совместно с встраиванием для задачи предсказания маскированных токенов.

6. Установите веса декодера равными весам встраивания.

7. Возвращайте результаты предсказания маскированных токенов и классификации.

В случае затруднений можно обратиться к:
https://github.com/graykode/nlp-tutorial/blob/master/5-2.BERT/BERT.py
"""


class BERT(nn.Module):
    # НАЧАЛО ВАШЕГО РЕШЕНИЯ
    raise NotImplementedError() # удалить эту строку в процессе решения
    # КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [ ]:
# Параметры BERT
maxlen = 30  # максимальная длина
batch_size = 6
max_pred = 5  # максимальное количество предсказываемых токенов
n_layers = 6  # количество слоев в кодировщике
n_heads = 12  # количество голов в Multi-Head Attention
d_model = 768  # размер вложения
d_ff = 768 * 4  # 4*d_model, размер скрытого слоя FeedForward
d_k = d_v = 64  # размерности K(=Q), V
n_segments = 2

text = (
    'Привет, как дела? Я - Ромео.\n'
    'Привет, Ромео. Меня зовут Джульетта. Приятно познакомиться.\n'
    'Приятно познакомиться. Как твои дела сегодня?\n'
    'Здорово. Моя бейсбольная команда выиграла соревнование.\n'
    'Поздравляю, Джульетта.\n'
    'Спасибо, Ромео'
)
sentences = re.sub("[.,!?\\-]", '', text.lower()).split('\n')  # фильтруем '.', ',', '?', '!'
word_list = list(set(" ".join(sentences).split()))
word_dict = {'[PAD]': 0, '[CLS]': 1, '[SEP]': 2, '[MASK]': 3}
for i, w in enumerate(word_list):
    word_dict[w] = i + 4
number_dict = {i: w for i, w in enumerate(word_dict)}
vocab_size = len(word_dict)

token_list = list()
for sentence in sentences:
    arr = [word_dict[s] for s in sentence.split()]
    token_list.append(arr)

model = BERT()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

batch = make_batch()
input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(*batch))

for epoch in range(100):
    optimizer.zero_grad()
    logits_lm, logits_clsf = model(input_ids, segment_ids, masked_pos)
    loss_lm = criterion(logits_lm.transpose(1, 2), masked_tokens)  # для маскированной модели языка
    loss_lm = (loss_lm.float()).mean()
    loss_clsf = criterion(logits_clsf, isNext)  # для классификации предложений
    loss = loss_lm + loss_clsf
    if (epoch + 1) % 10 == 0:
        print('Эпоха:', '%04d' % (epoch + 1), 'функция потерь =', '{:.6f}'.format(loss))
    loss.backward()
    optimizer.step()

# Предсказание маскированных токенов и isNext
input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(batch[0]))
print(text)
print([number_dict[w.item()] for w in input_ids[0] if number_dict[w.item()] != '[PAD]'])

logits_lm, logits_clsf = model(input_ids, segment_ids, masked_pos)
logits_lm = logits_lm.data.max(2)[1][0].data.numpy()
print('Список маскированных токенов: ', [pos.item() for pos in masked_tokens[0] if pos.item() != 0])
print('Предсказанный список маскированных токенов: ', [pos for pos in logits_lm if pos != 0])

logits_clsf = logits_clsf.data.max(1)[1].data.numpy()[0]
print('isNext : ', True if isNext else False)
print('Предсказано isNext : ', True if logits_clsf else False)

In [ ]:
# Импорт необходимых библиотек
from torch.utils.data import DataLoader, Dataset
from transformers import AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pandas as pd

# Загрузка датасета "20 Newsgroups"
from sklearn.datasets import fetch_20newsgroups
newsgroups_data = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
df = pd.DataFrame({'text': newsgroups_data.data, 'label': newsgroups_data.target})

# Разделение данных на обучающий и тестовый наборы
train_data, test_data, train_labels, test_labels = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)

In [ ]:
"""
**Задание 7 (3 балла): Реализация модели BERT из библиотеки transformers**

**Цель:** Ваша задача - реализовать модель BERT с использованием библиотеки transformers на наборе данных 20newsgroups.
Вывести на печать метрики модели на тестовом наборе test_data с помощью classification_report

В случае затруднений можно обратиться к:
https://huggingface.co/docs/transformers/model_doc/bert
"""

# Загрузка предварительно обученного BERT
# НАЧАЛО ВАШЕГО РЕШЕНИЯ
raise NotImplementedError() # удалить эту строку в процессе решения
# КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [ ]:
"""
**Задание 8 (3 балла): Реализация модели BART из библиотеки transformers**

**Цель:** Ваша задача - реализовать модель BART с использованием библиотеки transformers на наборе данных 20newsgroups.
Вывести на печать метрики модели на тестовом наборе test_data с помощью classification_report

В случае затруднений можно обратиться к:
https://huggingface.co/docs/transformers/model_doc/bart
"""

# Загрузка предварительно обученного BART
# НАЧАЛО ВАШЕГО РЕШЕНИЯ
raise NotImplementedError() # удалить эту строку в процессе решения
# КОНЕЦ ВАШЕГО РЕШЕНИЯ